<a href="https://colab.research.google.com/github/AnthonyAngatia/ML-Group-Final-Task/blob/main/House_Pricing_Regression_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, RobustScaler, PowerTransformer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn import linear_model
from sklearn.linear_model import Lasso,LassoCV
from keras.models import Sequential
from keras.layers import Dense, InputLayer
from keras.optimizers import Adam


# Load data

In [ ]:
data = pd.read_csv('Housing Pricing.csv')
data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,5,1999,2000,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,Unf,0,Unf,0,953,953,GasA,Ex,Y,SBrkr,953,694,0,1647,0,0,2,1,3,1,TA,7,Typ,1,TA,Attchd,1999.0,RFn,2,460,TA,TA,Y,0,40,0,0,0,0,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,1Story,6,6,1978,1988,Gable,CompShg,Plywood,Plywood,Stone,119.0,TA,TA,CBlock,Gd,TA,No,ALQ,790,Rec,163,589,1542,GasA,TA,Y,SBrkr,2073,0,0,2073,1,0,2,0,3,1,TA,7,Min1,2,TA,Attchd,1978.0,Unf,2,500,TA,TA,Y,349,0,0,0,0,0,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,9,1941,2006,Gable,CompShg,CemntBd,CmentBd,None,0.0,Ex,Gd,Stone,TA,Gd,No,GLQ,275,Unf,0,877,1152,GasA,Ex,Y,SBrkr,1188,1152,0,2340,0,0,2,0,4,1,Gd,9,Typ,2,Gd,Attchd,1941.0,RFn,1,252,TA,TA,Y,0,60,0,0,0,0,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,6,1950,1996,Hip,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,TA,TA,Mn,GLQ,49,Rec,1029,0,1078,GasA,Gd,Y,FuseA,1078,0,0,1078,1,0,1,0,2,1,Gd,5,Typ,0,NaN,Attchd,1950.0,Unf,1,240,TA,TA,Y,366,0,112,0,0,0,0,4,2010,WD,Normal,142125


Check the shape of the dataset

In [ ]:
data.shape

(1460, 77)

In [ ]:
# Drop the ID column as it is unique and not useful
data = data.drop( columns=['Id'] )
data.shape


(1460, 76)

## Check which columns have missing values

Analyse the number of the missing values and fill them or drop the columns

In [ ]:
null_values = data.isnull().sum()
null_values

MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
Street             0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 76, dtype: int64

In [ ]:
null_columns = data.isnull().any()
null_columns = data.columns[null_columns].tolist() #converting to a list
null_columns

['LotFrontage',
 'MasVnrType',
 'MasVnrArea',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu',
 'GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageQual',
 'GarageCond']

In [ ]:
data['FireplaceQu'] = data['FireplaceQu'].fillna(data['FireplaceQu'].mode()[0])
for column_name in null_columns:
  # The idea is to replace the null values in each column with the mode value
  # data[column_name] = data[column_name].fillna(data[column_name].mode()[0])
  # data[column_name] = data[column_name].fillna(method='ffill', inplace=True)
  data[column_name].fillna(method='bfill',inplace=True)

data.isnull().any().any() #checks if there is any missing value


False

# Encoding Dataset


In [ ]:
total_features = list(data.columns)
len(total_features)
categorical_features = list(data.select_dtypes(include=['object']))
len(categorical_features)


39

In [ ]:
numerical_list = list(set(total_features) - set(categorical_features))
len(numerical_list)

37

To do One Hot encoding, we need to categorize our data as nominal and ordinal 

In [ ]:
#Create a list of ordinal and nominal
nominal=['MSZoning','LandContour','LotConfig','Neighborhood']
ordinal =list(set(categorical_features) - set(nominal))

Perform Integer encoding for nominal data

In [ ]:
nominal_dataframe = pd.get_dummies(data[nominal])
nominal_dataframe

,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker
0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1456,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1457,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1458,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


Perform One Hot Encoding on ordinal data

In [ ]:
 #data[ordinal]
for feature in ordinal:
  data[feature]=data[feature].astype('category').cat.codes

ordinal_dataframe=data[ordinal]
ordinal_dataframe



,Heating,Exterior2nd,GarageFinish,BsmtExposure,KitchenQual,LotShape,SaleCondition,MasVnrType,Exterior1st,GarageCond,Foundation,Street,Condition2,SaleType,HeatingQC,ExterQual,HouseStyle,LandSlope,GarageType,BsmtCond,Electrical,PavedDrive,Utilities,ExterCond,BldgType,CentralAir,BsmtFinType2,RoofMatl,RoofStyle,FireplaceQu,GarageQual,Condition1,BsmtFinType1,BsmtQual,Functional
0,1,13,1,3,2,3,4,1,12,4,2,1,2,8,0,2,5,0,1,3,4,2,0,4,0,1,5,1,1,2,4,2,2,2,6
1,1,8,1,1,3,3,4,2,8,4,1,1,2,8,0,3,2,0,1,3,4,2,0,4,0,1,5,1,1,4,4,1,0,2,6
2,1,13,1,2,2,0,4,1,12,4,2,1,2,8,0,2,5,0,1,3,4,2,0,4,0,1,5,1,1,4,4,2,2,2,6
3,1,15,2,3,2,0,0,2,13,4,0,1,2,8,2,3,5,0,5,1,4,2,0,4,0,1,5,1,1,2,4,2,0,3,6
4,1,13,1,0,2,0,4,1,12,4,2,1,2,8,0,2,5,0,1,3,4,2,0,4,0,1,5,1,1,4,4,2,2,2,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1,13,1,3,3,3,4,2,12,4,2,1,2,8,0,3,5,0,1,3,4,2,0,4,0,1,5,1,1,4,4,2,5,2,6
1456,1,10,2,3,3,3,4,3,9,4,1,1,2,8,4,3,2,0,1,3,4,2,0,4,0,1,4,1,1,4,4,2,0,2,2
1457,1,5,1,3,2,3,4,2,5,4,4,1,2,8,0,0,5,0,1,1,4,2,0,2,0,1,5,1,1,2,4,2,2,3,6
1458,1,8,2,2,2,3,4,2,8,4,1,1,2,8,2,3,2,0,1,3,0,2,0,4,0,1,4,1,3,2,4,2,2,3,6


In [ ]:
new_encoded_dataframe = pd.concat([nominal_dataframe, ordinal_dataframe,data[numerical_list]], axis=1)

In [ ]:
new_encoded_dataframe['SalePrice']

0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1460, dtype: int64

# Feature Preprocessing
## Feature scaling and transformation

In [ ]:
target = ['SalePrice']
y = data[target]
X = new_encoded_dataframe.drop('SalePrice', axis=1)
X_std = StandardScaler().fit_transform(X)
y = StandardScaler().fit_transform(y)


# Feature Selection
We are doing feature selection using lasso regression

In [ ]:
# sel_ = SelectFromModel(LogisticRegression(C=1, penalty='l1', solver='liblinear'))
# sel_.fit(X_train_std, y_train)
# sel_.get_support()

In [ ]:
lasso_regressor = LassoCV()
lasso_regressor.fit(X_std, y)
lasso_regressor.coef_




/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:1088: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([-2.07106531e-02,  8.48692402e-03,  0.00000000e+00,  0.00000000e+00,
       -0.00000000e+00, -3.51902576e-02,  1.82262384e-02, -0.00000000e+00,
        0.00000000e+00, -0.00000000e+00,  2.11814182e-02, -1.81007659e-02,
       -1.28943400e-02,  0.00000000e+00,  1.83655766e-04, -0.00000000e+00,
        9.82928147e-05,  3.34160668e-03,  3.95370161e-04,  0.00000000e+00,
        4.68640611e-02, -3.16063467e-02, -6.17980083e-03, -6.91835879e-04,
        0.00000000e+00, -7.41757305e-03, -2.41841153e-03,  1.43616220e-03,
       -1.64818891e-02,  1.07691863e-01,  1.09722786e-01, -2.47831596e-02,
       -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,  3.22403111e-02,
        7.98789435e-02,  0.00000000e+00,  1.88707599e-02, -5.34821103e-04,
       -0.00000000e+00, -1.88772533e-02, -4.30587795e-02, -7.84209526e-02,
        0.00000000e+00,  2.96425107e-02,  1.81673591e-02, -1.42095598e-02,
        1.06312418e-02,  0.00000000e+00,  7.47036694e-03, -1.89697422e-02,
       -1.24310708e-02, -

In [ ]:
coef_columns = pd.Series(lasso_regressor.coef_,index=X.columns)
# print(coef_columns)
sorted_coef = coef_columns.sort_values()
print(sorted_coef)

BsmtQual               -0.080150
KitchenQual            -0.078421
MSSubClass             -0.060397
BldgType               -0.053739
ExterQual              -0.053699
                          ...   
GarageCars              0.086118
Neighborhood_NoRidge    0.107692
Neighborhood_NridgHt    0.109723
OverallQual             0.166489
GrLivArea               0.262790
Length: 110, dtype: float64


In [ ]:
selected_features = sorted_coef[coef_columns != 0]
no_of_selected_features = sum(coef_columns != 0)
# no_of_selected_features
rejected_features = sorted_coef[coef_columns == 0]
no_of_rejected_features = sum(coef_columns == 0)
total_features = no_of_selected_features + no_of_rejected_features
print("L1 selected only ",no_of_selected_features," features out of ",total_features," from the transformed dataset ")

L1 selected only  82  features out of  110  from the transformed dataset 


In [ ]:
rejected_features.index

Index(['GarageType', 'Electrical', 'LotConfig_Inside', 'PoolArea',
       'MSZoning_RM', 'Condition1', 'Neighborhood_Blueste', 'Foundation',
       'GarageQual', 'GarageYrBlt', 'MSZoning_RL', 'MSZoning_RH',
       'LandContour_Lvl', 'Neighborhood_MeadowV', 'Neighborhood_CollgCr',
       '2ndFlrSF', 'LotConfig_Corner', 'Neighborhood_SWISU',
       'Neighborhood_Sawyer', 'LotShape', 'Neighborhood_SawyerW',
       'LandContour_Low', 'Neighborhood_Timber', 'EnclosedPorch',
       'BsmtFinType2', 'Exterior2nd', 'GarageArea', 'BsmtUnfSF'],
      dtype='object')

In [ ]:
# Creating a new dataframe of the standardized features without the rejected features
# 
X_selected_df = pd.DataFrame(X_std, columns = X.columns)
X_selected_df = X_selected_df.drop(rejected_features.index, axis=1)
X_selected_df

,MSZoning_C (all),MSZoning_FV,LandContour_Bnk,LandContour_HLS,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,Neighborhood_Blmngtn,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Veenker,Heating,GarageFinish,BsmtExposure,KitchenQual,SaleCondition,MasVnrType,Exterior1st,GarageCond,Street,Condition2,SaleType,HeatingQC,ExterQual,HouseStyle,LandSlope,...,Utilities,ExterCond,BldgType,CentralAir,RoofMatl,RoofStyle,FireplaceQu,BsmtFinType1,BsmtQual,Functional,1stFlrSF,BsmtHalfBath,GarageCars,BedroomAbvGr,MoSold,LotArea,LowQualFinSF,BsmtFullBath,YearBuilt,ScreenPorch,GrLivArea,YrSold,KitchenAbvGr,TotalBsmtSF,HalfBath,BsmtFinSF2,MSSubClass,FullBath,OverallCond,LotFrontage,OverallQual,MiscVal,BsmtFinSF1,YearRemodAdd,MasVnrArea,WoodDeckSF,Fireplaces,3SsnPorch,OpenPorchSF,TotRmsAbvGrd
0,-0.083045,-0.215859,-0.21236,-0.188311,-0.262324,-0.182380,-0.052414,-0.10854,-0.105263,-0.203395,-0.139832,-0.190252,-0.271163,-0.239176,-0.16125,-0.186352,-0.426833,-0.078757,-0.229416,-0.169981,-0.235958,-0.289638,-0.250182,-0.131991,-0.087129,-0.123046,-0.230122,0.640462,-0.409369,0.208502,-1.238042,0.743092,0.187973,0.064238,-0.03174,0.313867,-0.884658,-0.777976,1.026689,-0.225716,...,-0.02618,0.364207,-0.411691,0.263813,-0.125797,-0.491516,-0.430030,-0.407795,-0.290552,0.255978,-0.793434,-0.241061,0.311725,0.163779,-1.599111,-0.207142,-0.120242,1.107810,1.050994,-0.270208,0.370333,0.138777,-0.211454,-0.459303,1.227585,-0.288653,0.073375,0.789741,-0.517200,-0.221669,0.651479,-0.087688,0.575425,0.878668,0.509688,-0.752176,-0.951226,-0.116339,0.216503,0.912210
1,-0.083045,-0.215859,-0.21236,-0.188311,-0.262324,5.483049,-0.052414,-0.10854,-0.105263,-0.203395,-0.139832,-0.190252,-0.271163,-0.239176,-0.16125,-0.186352,-0.426833,-0.078757,-0.229416,-0.169981,-0.235958,-0.289638,-0.250182,-0.131991,11.477250,-0.123046,-0.230122,-1.104081,0.795629,0.208502,0.388908,-0.508251,0.187973,0.064238,-0.03174,0.313867,-0.884658,0.663451,-0.543457,-0.225716,...,-0.02618,0.364207,-0.411691,0.263813,-0.125797,-0.491516,1.792425,-1.505270,-0.290552,0.255978,0.257140,3.948809,0.311725,0.163779,-0.489110,-0.091886,-0.120242,-0.819964,0.156734,-0.270208,-0.482512,-0.614439,-0.211454,0.466465,-0.761621,-0.288653,-0.872563,0.789741,2.179628,0.404967,-0.071836,-0.087688,1.171992,-0.429577,-0.572415,1.626195,0.600495,-0.116339,-0.704483,-0.318683
2,-0.083045,-0.215859,-0.21236,-0.188311,-0.262324,-0.182380,-0.052414,-0.10854,-0.105263,-0.203395,-0.139832,-0.190252,-0.271163,-0.239176,-0.16125,-0.186352,-0.426833,-0.078757,-0.229416,-0.169981,-0.235958,-0.289638,-0.250182,-0.131991,-0.087129,-0.123046,-0.230122,-0.231809,-0.409369,0.208502,-1.238042,0.743092,0.187973,0.064238,-0.03174,0.313867,-0.884658,-0.777976,1.026689,-0.225716,...,-0.02618,0.364207,-0.411691,0.263813,-0.125797,-0.491516,1.792425,-0.407795,-0.290552,0.255978,-0.627826,-0.241061,0.311725,0.163779,0.990891,0.073480,-0.120242,1.107810,0.984752,-0.270208,0.515013,0.138777,-0.211454,-0.313369,1.227585,-0.288653,0.073375,0.789741,-0.517200,-0.096342,0.651479,-0.087688,0.092907,0.830215,0.321976,-0.752176,0.600495,-0.116339,-0.070361,-0.318683
3,-0.083045,-0.215859,-0.21236,-0.188311,-0.262324,-0.182380,-0.052414,-0.10854,-0.105263,-0.203395,-0.139832,5.256182,-0.271163,-0.239176,-0.16125,-0.186352,-0.426833,-0.078757,-0.229416,-0.169981,-0.235958,-0.289638,-0.250182,-0.131991,-0.087129,-0.123046,1.000568,0.640462,-0.409369,-3.426284,0.388908,1.055928,0.187973,0.064238,-0.03174,0.313867,0.265476,0.663451,1.026689,-0.225716,...,-0.02618,0.364207,-0.411691,0.263813,-0.125797,-0.491516,-0.430030,-1.505270,0.852861,0.255978,-0.521734,-0.241061,1.650307,0.163779,-1.599111,-0.096897,-0.120242,1.107810,-1.863632,-0.270208,0.383659,-1

## Buld and Train the model


In [ ]:
print(X_selected_df.to_numpy().shape)

(1460, 82)


In [ ]:
# standardize the data
X_selected_array = X_selected_df.to_numpy()
print(X_selected_array.shape)
# y_array = y.to_numpy()
print(y_array.shape)

(1460, 82)
(1460, 1)


## Metrics Used

We are using MSE to measure the accuracy of our model. The lower the MSE the better the model.

In [ ]:
input_size = X_selected_array.shape[1]
model=Sequential()
model.add(Dense(128, input_dim=input_size, kernel_initializer='normal', activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
# model.add(Dense(4, activation='relu'))
# model.add(Dense(2, activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))
# model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy', 'MSE'])
model.compile(Adam(learning_rate=0.01), loss='mean_squared_error', metrics=['MSE'])

In [ ]:
model.summary()

Model: "sequential_43"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_181 (Dense)            (None, 128)               10624     
_________________________________________________________________
dense_182 (Dense)            (None, 64)                8256      
_________________________________________________________________
dense_183 (Dense)            (None, 32)                2080      
_________________________________________________________________
dense_184 (Dense)            (None, 16)                528       
_________________________________________________________________
dense_185 (Dense)            (None, 8)                 136       
_________________________________________________________________
dense_186 (Dense)            (None, 1)                 9         
Total params: 21,633
Trainable params: 21,633
Non-trainable params: 0
_________________________________________________

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_selected_array, y, test_size=0.3, random_state=42)

In [ ]:
history = model.fit(X_train, y_train, verbose = 1
                    , epochs=100, validation_data=(X_test, y_test))

Epoch 1/100
32/32 [==============================] - 0s 4ms/step - loss: 0.0350 - MSE: 0.0350 - val_loss: 0.1201 - val_MSE: 0.1201
Epoch 2/100
32/32 [==============================] - 0s 3ms/step - loss: 0.0310 - MSE: 0.0310 - val_loss: 0.1226 - val_MSE: 0.1226
Epoch 3/100
32/32 [==============================] - 0s 3ms/step - loss: 0.0302 - MSE: 0.0302 - val_loss: 0.1291 - val_MSE: 0.1291
Epoch 4/100
32/32 [==============================] - 0s 3ms/step - loss: 0.0234 - MSE: 0.0234 - val_loss: 0.1186 - val_MSE: 0.1186
Epoch 5/100
32/32 [==============================] - 0s 3ms/step - loss: 0.0352 - MSE: 0.0352 - val_loss: 0.1608 - val_MSE: 0.1608
Epoch 6/100
32/32 [==============================] - 0s 3ms/step - loss: 0.0430 - MSE: 0.0430 - val_loss: 0.1408 - val_MSE: 0.1408
Epoch 7/100
32/32 [==============================] - 0s 3ms/step - loss: 0.0313 - MSE: 0.0313 - val_loss: 0.1226 - val_MSE: 0.1226
Epoch 8/100
32/32 [==============================] - 0s 3ms/step - loss: 0.0224 - M

In [ ]:
historydf=pd.DataFrame(history.history, index=history.epoch)
historydf

,loss,MSE,val_loss,val_MSE
0,0.035037,0.035037,0.120069,0.120069
1,0.031007,0.031007,0.122573,0.122573
2,0.030233,0.030233,0.129052,0.129052
3,0.023354,0.023354,0.118568,0.118568
4,0.035191,0.035191,0.160783,0.160783
...,...,...,...,...
95,0.003676,0.003676,0.117804,0.117804
96,0.002330,0.002330,0.116989,0.116989
97,0.002720,0.002720,0.120170,0.120170
98,0.003442,0.003442,0.114907,0.114907
